In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
df1_7=pd.read_parquet('./data/train/1.회원정보/201807_train_회원정보.parquet')
df2_7=pd.read_parquet('./data/train/2.신용정보/201807_train_신용정보.parquet')
df1_8=pd.read_parquet('./data/train/1.회원정보/201808_train_회원정보.parquet')
df2_8=pd.read_parquet('./data/train/2.신용정보/201808_train_신용정보.parquet')

In [3]:
df1_9=pd.read_parquet('./data/train/1.회원정보/201809_train_회원정보.parquet')
df2_9=pd.read_parquet('./data/train/2.신용정보/201809_train_신용정보.parquet')
df1_10=pd.read_parquet('./data/train/1.회원정보/201810_train_회원정보.parquet')
df2_10=pd.read_parquet('./data/train/2.신용정보/201810_train_신용정보.parquet')

In [4]:
df1_11=pd.read_parquet('./data/train/1.회원정보/201811_train_회원정보.parquet')
df2_11=pd.read_parquet('./data/train/2.신용정보/201811_train_신용정보.parquet')
df1_12=pd.read_parquet('./data/train/1.회원정보/201812_train_회원정보.parquet')
df2_12=pd.read_parquet('./data/train/2.신용정보/201812_train_신용정보.parquet')

## 1. 6개월치 1,2 데이터프레임 merge

In [5]:
# 병합 대상 월 리스트
months = ['7', '8', '9', '10', '11', '12']

# 병합된 데이터를 담을 리스트
merged_list = []

for m in months:
    df1 = globals()[f'df1_{m}']
    df2 = globals()[f'df2_{m}']
    
    # ID와 기준년월 기준 병합
    merged = pd.merge(df1, df2, on=['ID', '기준년월'], how='inner')
    merged_list.append(merged)

# 병합된 모든 월 데이터를 하나로 합치기
final_df = pd.concat(merged_list, ignore_index=True)

print("모든 월 데이터 병합 완료! 최종 shape:", final_df.shape)

모든 월 데이터 병합 완료! 최종 shape: (2400000, 118)


In [6]:
final_df

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,연체감액여부_R3M,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0회,0,3,3,0,0,0,0,99999999
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0회,0,3,3,0,0,0,0,322
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0회,0,3,3,0,0,0,0,2378
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0회,0,3,3,0,0,0,0,99999999
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0회,0,3,3,0,0,0,0,99999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,70대이상,E,1,1,1,1,1,...,0,0회,0,3,3,0,0,0,0,99999999
2399996,201812,TRAIN_399996,2,50대,D,1,1,1,1,1,...,0,0회,0,3,3,0,0,0,0,99999999
2399997,201812,TRAIN_399997,1,30대,C,1,1,0,1,1,...,0,0회,0,3,3,0,0,0,0,99999999
2399998,201812,TRAIN_399998,1,40대,E,1,1,1,1,1,...,0,0회,0,3,3,0,0,0,0,99999999


## 2. 데이터 전처리

#### datetime 변환

In [6]:
# 확인하고 싶은 컬럼 리스트
date_columns = [
    '기준년월',
    '입회일자_신용',
    '최종유효년월_신용_이용가능',
    '최종유효년월_신용_이용',
    '최종카드발급일자',
    'RV신청일자',
    '최종이용일자_기본',
    '최종이용일자_신판',
    '최종이용일자_CA',
    '최종이용일자_카드론',
    '최종이용일자_체크',
    '최종이용일자_일시불',
    '최종이용일자_할부',
    '최종카드론_대출일자'
]

# 실제 존재하는 컬럼만 추출
existing_columns = [col for col in date_columns if col in final_df.columns]

# 결과 출력
print("final_df에 존재하는 날짜 관련 컬럼:")
print(existing_columns)

final_df에 존재하는 날짜 관련 컬럼:
['기준년월', '입회일자_신용', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', 'RV신청일자']


In [7]:
# 날짜 컬럼 리스트 (입회일자 및 최종이용일자 관련)
date_cols = ['기준년월', '입회일자_신용', '최종유효년월_신용_이용가능', 
             '최종유효년월_신용_이용', '최종카드발급일자', 'RV신청일자']
# final_df에 존재하는 컬럼만 선택
existing_date_cols = [col for col in date_cols if col in final_df.columns]
# 각 컬럼에 대해 이상치 처리 및 라벨 인코딩 수행
for col in existing_date_cols:
    # 문자열로 변환하고 이상치 '10101'을 '0'으로 대체
    final_df[col] = final_df[col].astype(str).replace('10101', '0')
    # 라벨 인코딩
    le = LabelEncoder()
    final_df[col] = le.fit_transform(final_df[col])

#### 고유값이 하나인 범주형데이터_피처 삭제

In [8]:
# 데이터 타입이 object이거나 category인 컬럼 추출
categorical_columns = final_df.select_dtypes(include=['object', 'category']).columns.tolist()

print("범주형 변수 목록:")
print(categorical_columns)

범주형 변수 목록:
['ID', '연령', 'Segment', '가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage', '자발한도감액횟수_R12M', '한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부', '한도심사요청건수']


In [9]:
# 범주형 변수 추출
categorical_columns = final_df.select_dtypes(include=['object', 'category']).columns

# 유일한 값이 하나만 존재하는 컬럼 찾기
single_value_columns = [col for col in categorical_columns if final_df[col].nunique(dropna=False) == 1]

print("값이 하나뿐인 범주형 변수:")
print(single_value_columns)

값이 하나뿐인 범주형 변수:
['상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']


In [10]:
final_df = final_df.drop(columns=['상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M'])

#### 결측치 처리

In [11]:
# 결측치가 있는 칼럼과 결측치 수
null_columns = final_df.isnull().sum()
null_columns = null_columns[null_columns > 0].sort_values(ascending=False)

# 보기 좋게 DataFrame으로 출력
null_df = null_columns.reset_index()
null_df.columns = ['Column', 'MissingValues']

print(null_df)

       Column  MissingValues
0  _2순위신용체크구분         958115
1    가입통신회사코드         387570
2       직장시도명         244969
3    RV전환가능여부          29473
4  _1순위신용체크구분          27950


In [18]:
final_df['rv최초시작후경과일'].dtype

dtype('int64')

In [19]:
# 결측치를 지정된 값으로 채우거나 dropna 처리
final_df['_2순위신용체크구분'] = final_df['_2순위신용체크구분'].fillna('없음')
final_df['가입통신회사코드'] = final_df['가입통신회사코드'].fillna('그외')
final_df['직장시도명'] = final_df['직장시도명'].fillna('없음')
final_df['rv최초시작후경과일'] = final_df['rv최초시작후경과일'].replace(99999999, 0)

# RV전환가능여부, _1순위신용체크구분 칼럼은 결측치를 가진 행을 제거
final_df = final_df.dropna(subset=['RV전환가능여부', '_1순위신용체크구분'])

# 처리 완료 후 결과 출력
print(final_df[['_2순위신용체크구분', '가입통신회사코드', '직장시도명', 'RV전환가능여부', '_1순위신용체크구분']].head())

  _2순위신용체크구분 가입통신회사코드 직장시도명 RV전환가능여부 _1순위신용체크구분
0         체크       L사    서울        N         신용
1         없음       K사    경기        Z         신용
2         체크       S사    서울        N         신용
3         체크       L사    부산        N         신용
4         체크       K사    광주        Z         신용


#### 범주형데이터 찾기(int or float -> category)

In [20]:
# 변환 대상 컬럼 리스트
category_cols = ['최종카드론_금융상환방식코드',
    '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA',
    '회원여부_이용가능_카드론', '소지여부_신용', '회원여부_연체',
    '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM',
    '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS',
    '마케팅동의여부', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인',
    '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인',
    '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인',
    '일시불ONLY전환가능여부', '연체감액여부_R3M', '시장단기연체여부_R6M',
    '시장단기연체여부_R3M', '청구서발송여부_B0', '청구서발송여부_R3M',
    '청구서발송여부_R6M'
]
# 실제 존재하는 컬럼만 선택
existing_cols = [col for col in category_cols if col in final_df.columns]
# 선택된 컬럼들을 category 타입으로 변환
final_df[existing_cols] = final_df[existing_cols].astype('category')

In [21]:
existing_cols

['남녀구분코드',
 '회원여부_이용가능',
 '회원여부_이용가능_CA',
 '회원여부_이용가능_카드론',
 '소지여부_신용',
 '회원여부_연체',
 '이용거절여부_카드론',
 '동의여부_한도증액안내',
 '수신거부여부_TM',
 '수신거부여부_DM',
 '수신거부여부_메일',
 '수신거부여부_SMS',
 '마케팅동의여부',
 '보유여부_해외겸용_본인',
 '이용가능여부_해외겸용_본인',
 '이용여부_3M_해외겸용_본인',
 '보유여부_해외겸용_신용_본인',
 '이용가능여부_해외겸용_신용_본인',
 '이용여부_3M_해외겸용_신용_본인',
 '일시불ONLY전환가능여부',
 '연체감액여부_R3M',
 '시장단기연체여부_R6M',
 '시장단기연체여부_R3M']

#### 범주형데이터 인코딩

In [22]:
print(categorical_columns)

Index(['ID', '연령', 'Segment', '가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분',
       '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M',
       '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage', '자발한도감액횟수_R12M',
       '한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부', '한도심사요청건수'],
      dtype='object')


In [23]:
one_hot_cols = ['남녀구분코드',
 '회원여부_이용가능',
 '회원여부_이용가능_CA',
 '회원여부_이용가능_카드론',
 '소지여부_신용',
 '회원여부_연체',
 '이용거절여부_카드론',
 '동의여부_한도증액안내',
 '수신거부여부_TM',
 '수신거부여부_DM',
 '수신거부여부_메일',
 '수신거부여부_SMS',
 '마케팅동의여부',
 '보유여부_해외겸용_본인',
 '이용가능여부_해외겸용_본인',
 '이용여부_3M_해외겸용_본인',
 '보유여부_해외겸용_신용_본인',
 '이용가능여부_해외겸용_신용_본인',
 '이용여부_3M_해외겸용_신용_본인',
 '일시불ONLY전환가능여부',
 '연체감액여부_R3M',
 '시장단기연체여부_R6M']
final_df = pd.get_dummies(final_df, columns=one_hot_cols, prefix=one_hot_cols)

In [24]:
# 인코딩 대상 컬럼
one_hot_cols = [
'가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분',  'Life_Stage', 
'한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부'
]

label_encode_cols = [
'연회비발생카드수_B0M', 
'자발한도감액횟수_R12M',
 '한도심사요청건수'
]

# 1. 원핫 인코딩
final_df = pd.get_dummies(final_df, columns=one_hot_cols, prefix=one_hot_cols)

# 2. 라벨 인코딩 (올인원 스타일)
le = LabelEncoder()
for col in label_encode_cols:
    final_df[col] = final_df[col].fillna('NaN')  # 결측치 임시 문자열 처리
    final_df[col] = le.fit_transform(final_df[col].astype(str))

# 결과 확인
print("인코딩 완료. final_df shape:", final_df.shape)

인코딩 완료. final_df shape: (2365002, 184)


In [25]:
# 라벨 인코더 선언
le = LabelEncoder()

# 결측치가 있다면 임시 문자열로 대체
final_df['Segment'] = final_df['Segment'].fillna('NaN')

# 인코딩 수행 (기존 Segment 칼럼 덮어쓰기)
final_df['Segment'] = le.fit_transform(final_df['Segment'])

# 인코딩된 값 확인
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("라벨 인코딩 매핑 결과:")
for k, v in label_mapping.items():
    print(f"{k} → {v}")

라벨 인코딩 매핑 결과:
A → 0
B → 1
C → 2
D → 3
E → 4


In [26]:
final_df

,기준년월,ID,연령,Segment,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,입회경과개월수_신용,탈회횟수_누적,최종탈회후경과월,...,Life_Stage_자녀독립기,Life_Stage_자녀성장(1),Life_Stage_자녀성장(2),Life_Stage_자녀출산기,한도증액횟수_R12M_0회,한도증액횟수_R12M_1회이상,카드론동의여부_N,카드론동의여부_Y,RV전환가능여부_N,RV전환가능여부_Z
0,0,TRAIN_000000,40대,3,1,1,259,67,1,61,...,False,False,True,False,True,False,False,True,True,False
1,0,TRAIN_000001,30대,4,1,1,314,12,1,98,...,False,True,False,False,True,False,False,True,False,True
2,0,TRAIN_000002,30대,2,1,1,202,124,1,60,...,False,False,False,True,True,False,False,True,True,False
3,0,TRAIN_000003,40대,3,2,2,299,27,1,100,...,False,False,True,False,True,False,False,True,True,False
4,0,TRAIN_000004,40대,4,1,1,324,2,1,101,...,False,True,False,False,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,5,TRAIN_399995,70대이상,4,1,1,121,209,0,0,...,False,False,False,False,True,False,False,True,False,True
2399996,5,TRAIN_399996,50대,3,1,1,313,17,1,50,...,False,False,True,False,False,True,False,True,False,True
2399997,5,TRAIN_399997,30대,2,1,1,215,115,0,0,...,False,False,False,True,True,False,False,True,True,False
2399998,5,TRAIN_399998,40대,4,1,1,259,71,0,0,...,False,True,False,False,True,False,False,True,False,True


## 3. Segment와 상관계수 도출

In [27]:
# 제외할 칼럼 목록
exclude_cols = ['ID', '기준년월']

# 상관계수 계산 (Segment 기준)
correlations = final_df.drop(columns=exclude_cols).corr(numeric_only=True)['Segment'].drop('Segment')

# 상관계수 0.1 이상인 피처 추출
high_corr_features = correlations[correlations.abs() >= 0.1]

# 결과 출력
print("Segment와 상관계수(|r| ≥ 0.1)인 피처 수:", len(high_corr_features))
print("\n해당 피처들:\n", high_corr_features.sort_values(ascending=False))

Segment와 상관계수(|r| ≥ 0.1)인 피처 수: 52

해당 피처들:
 입회일자_신용                 0.251962
이용여부_3M_해외겸용_본인_0       0.226255
이용여부_3M_해외겸용_신용_본인_0    0.219731
_2순위신용체크구분_없음           0.193600
최종유효년월_신용_이용            0.173156
이용가능여부_해외겸용_신용_본인_0     0.164885
이용가능여부_해외겸용_본인_0        0.153112
보유여부_해외겸용_신용_본인_0       0.141780
보유여부_해외겸용_본인_0          0.136853
수신거부여부_TM_0             0.106607
수신거부여부_메일_0             0.106102
수신거부여부_DM_0             0.104415
직장시도명_없음                0.101037
수신거부여부_DM_1            -0.104415
수신거부여부_메일_1            -0.106102
최초한도금액                 -0.106429
수신거부여부_TM_1            -0.106607
CA이자율_할인전              -0.117925
보유여부_해외겸용_본인_1         -0.136853
보유여부_해외겸용_신용_본인_1      -0.141780
이용가능카드수_체크             -0.144917
유효카드수_체크               -0.145294
이용가능여부_해외겸용_본인_1       -0.153112
이용카드수_체크               -0.154316
이용가능여부_해외겸용_신용_본인_1    -0.164885
이용금액_R3M_체크            -0.209004
이용가능카드수_신용_가족          -0.212495
유효카드수_신용_가족            -0.212545
이용카드수_신용_가족            -0.21423

In [28]:
# 저장할 컬럼 목록: 상관계수 높은 피처 + ID, 기준년월, Segment
selected_cols = ['ID', '기준년월', 'Segment'] + list(high_corr_features.index)

# 해당 컬럼만 추출
filtered_df = final_df[selected_cols]

# CSV로 저장
filtered_df.to_csv('./data/0.1이상 상관계수(12).csv', index=False, encoding='utf-8-sig')

print("0.1이상 상관계수(12).csv 파일로 저장 완료!")

0.1이상 상관계수(12).csv 파일로 저장 완료!
